In [1]:
!uv add anthropic

Resolved 119 packages in 2ms
Audited 35 packages in 0.29ms


In [2]:
import anthropic
from app.config import Settings
from typing import Dict

settings = Settings()


In [3]:
system="""
You are Glowbot, a smart skincare assistant who helps users create personalized skincare routines. 
Interview users naturally, asking one question at a time while maintaining a friendly, conversational tone. 
Focus on gathering essential information: age, skin type, current concerns, health considerations (pregnancy, medications, allergies), and current skincare routine.

Be concise in your responses, remembering this is a chat conversation. 
Progress through information gathering logically - start with basic skin information, then health safety checks, current routine assessment, and finally provide personalized recommendations. 
Don't move to recommendations until you have all necessary information. If users mention severe skin conditions, always recommend consulting a dermatologist.

When making recommendations, structure them into morning and evening routines with clear usage instructions. 
Include basic safety information like patch testing and potential product interactions. 
Keep responses brief but informative, and don't hesitate to ask clarifying questions when needed to ensure proper recommendations.
"""

In [4]:
class NaiveAgent:
    def __init__(self, settings: Settings):
        self.client = anthropic.Anthropic(api_key=settings.claude_api_key)
        self.history = []
        self.model = "claude-3-5-sonnet-20241022"
    
    def get_claude_response(self, message: str) -> str:
        self.history.append({
            "role": "user",
            "content": message
        })

        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            messages=self.history,
            system=system
        )

        assert response.content[0].type == "text"

        assistant_message = response.content[0].text
        self.history.append({
            "role": "assistant",
            "content": assistant_message
        })
        
        return assistant_message


now lets run my app

In [ ]:
conflow = NaiveAgent(settings)

while True:
    user_input = input("You: ")
    if not user_input:
        print("$ Please enter a valid input. Type '/exit' to end the conversation.")
        continue
    if user_input.lower() in ["/exit", "/quit"]:
        print("$ Exiting the conversation.")
        break
    print(f"> User: {user_input}")
    response = conflow.get_claude_response(user_input)
    print(f"> Glowbot: {response}")

$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.
$ Please enter a valid input. Type '/exit' to end the conversation.


In [3]:
from enum import Enum
from pydantic import BaseModel, Field

class InterviewAction(str, Enum):
    """Defines the possible actions the bot can take after processing user input"""
    ASK = "ask"  # Ask a new question
    FOLLOWUP = "followup"  # Ask for clarification or more details
    DONE = "done"  # Consultation complete

class InterviewMessage(BaseModel): 
    """Defines the structure of a chat message"""
    reflection: str = Field(
        description="A concise summary/reflection of the user's input. What facts and hints did you gather from the user's message?",
        min_length=10,
        max_length=500
    )

    action: InterviewAction = Field(
          description="The next action to take in the conversation"
    )

    message: str = Field(
        description="The message to send to the user",
        min_length=1,
        max_length=2000
    )

# Example prompt for Claude
PROMPT_TEMPLATE = """
Based on the user's message, provide a response in this structure:
{
    "reflection": "Brief summary of what I understood from the user",
    "action": "ask|followup|done",
    "message": "Your next message to the user"
}

User message: {user_message}
"""



In [ ]:
class StructuredAgent:
    system="""
You are Glowbot, a smart skincare assistant who helps users create personalized skincare routines. Your current goal is to gather essential information, that will help us provide personalized skincare recommendations LATER ON.
Interview users naturally, asking one question at a time while maintaining a friendly, conversational tone. 
Focus on gathering essential information: age, skin type, current concerns, health considerations (pregnancy, medications, allergies), and current skincare routine.

Be concise in your responses, remembering this is a chat conversation. 
Progress through information gathering logically - start with basic skin information, then health safety checks, current routine assessment, and finally provide personalized recommendations. 

Guidelines:
- Always respond in the same language as the user.
- Glowbot, as female.
- When user mentioned something that can indicate us an interesting information, use a follow-up questions based on specific responses.
- NEVER suggest a product or a routine in this conversation. Your GOAL is ONLY to gather information.
"""

    def __init__(self, settings: Settings):
        self.client = anthropic.Anthropic(api_key=settings.claude_api_key)
        self.history = []
        self.model = "claude-3-5-sonnet-20241022"

    
    def get_claude_response(self, message: str) -> InterviewMessage:
        # if len(self.history) > 0 and not isinstance(self.history[-1]["content"][0],str) and self.history[-1]["content"][0] == "tool_use":
        #     self.history.append({
        #         "role": "user",
        #         "content": [
        #             {
        #                 "type": "tool_result",
        #                 "tool_use_id": self.history[-1]["content"][0]["id"],
        #                 "content": message
        #             }
        #         ]
        #     })
        # else:
        #     self.history.append({
        #         "role": "user",
        #         "content": message
        #     })
        

        self.history.append({
            "role": "user",
            "content": message
        })

        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            messages=self.history,
            tools=[{"name": "output", "input_schema": InterviewMessage.model_json_schema()}],
            tool_choice={"name": "output", "type": "tool"},
            system=self.system
        )

        assert response.content[0].type == "tool_use"

        assistant_message = response.content[0].input
        resp = InterviewMessage.model_validate(assistant_message)
        # self.history.append({"role": "assistant", "content": response.content[0].model_dump()})
        # self.history.append({"role": "assistant", "content": resp.message})
        self.history.append({"role": "assistant", "content": resp.model_dump_json()})

        
        return resp

sflow = StructuredAgent(settings)

while True:
    user_input = input("You: ")
    if not user_input or user_input.lower() in ["/exit", "/quit"]:
        print("$ Exiting the conversation.")
        break
    print(f"> User: {user_input}")
    response = sflow.get_claude_response(user_input)
    print(f"DEBUG || {response}")
    if response.action == InterviewAction.DONE:
        print(f"!!DONE!! Glowbot: {response.message}")
        break
    print(f"> Glowbot: {response.message}")

In [ ]:
from enum import Enum
from pydantic import BaseModel, Field
import anthropic
from typing import Dict, List, Optional, Any

class InterviewAction(str, Enum):
    """Defines the possible actions the bot can take after processing user input"""
    ASK = "ask"  # Ask a new question
    FOLLOWUP = "followup"  # Ask for clarification or more details
    DONE = "done"  # Consultation complete

class InterviewMessage(BaseModel): 
    """Defines the structure of a chat message"""
    reflection: str = Field(
        description="A concise summary/reflection of the user's input. What facts and hints did you gather from the user's message?",
        min_length=10,
        max_length=500
    )

    action: InterviewAction = Field(
          description="The next action to take in the conversation"
    )

    message: str = Field(
        description="The message to send to the user",
        min_length=1,
        max_length=2000
    )
    
    # New field to track what information was collected
    collected_info: Dict[str, Any] = Field(
        default_factory=dict,
        description="Information collected from this exchange"
    )

class ConversationState(str, Enum):
    """Defines all possible states in the GlowBot conversation flow"""
    GREETING = "greeting"
    AGE_VERIFICATION = "age_verification"
    SKIN_TYPE = "skin_type"
    SKIN_CONCERNS = "skin_concerns"
    HEALTH_CHECK = "health_check"
    SUN_EXPOSURE = "sun_exposure"
    CURRENT_ROUTINE = "current_routine"
    PRODUCT_PREFERENCES = "product_preferences"
    SUMMARY = "summary"
    COMPLETE = "complete"

class HealthInfo(BaseModel):
    """Health-related information"""
    is_pregnant: Optional[bool] = None
    is_nursing: Optional[bool] = None
    planning_pregnancy: Optional[bool] = None
    medications: List[str] = []
    allergies: List[str] = []
    sensitivities: List[str] = []

class SkinProfile(BaseModel):
    """User's skin profile"""
    age_verified: Optional[bool] = None
    skin_type: Optional[str] = None  # dry, oily, combination, normal
    concerns: List[str] = []  # hyperpigmentation, aging, acne, etc.
    sun_exposure: Optional[str] = None  # minimal, moderate, high

class UserRoutine(BaseModel):
    """User's current skincare routine"""
    morning_cleanser: Optional[str] = None
    morning_treatments: List[str] = []
    morning_moisturizer: Optional[str] = None
    morning_sunscreen: Optional[str] = None
    evening_makeup_removal: Optional[str] = None
    evening_cleanser: Optional[str] = None
    evening_treatments: List[str] = []
    evening_moisturizer: Optional[str] = None

class UserPreferences(BaseModel):
    """User's product preferences"""
    budget_range: Optional[str] = None  # budget-friendly, mid-range, high-end
    requirements: List[str] = []  # vegan, cruelty-free, fragrance-free, etc.

class ConversationContext(BaseModel):
    """Maintains the context of the conversation"""
    user_id: str = "user"  # Default user ID
    state: ConversationState = ConversationState.GREETING
    language: str = "en"  # Default language is English
    health_info: HealthInfo = Field(default_factory=HealthInfo)
    skin_profile: SkinProfile = Field(default_factory=SkinProfile)
    routine: UserRoutine = Field(default_factory=UserRoutine)
    preferences: UserPreferences = Field(default_factory=UserPreferences)
    missing_info: List[str] = []
    
    def update(self, updates: Dict[str, Any]) -> None:
        """Update context with new information"""
        for key, value in updates.items():
            if hasattr(self, key):
                if isinstance(getattr(self, key), BaseModel) and isinstance(value, dict):
                    # Update nested models
                    for subkey, subvalue in value.items():
                        if hasattr(getattr(self, key), subkey):
                            setattr(getattr(self, key), subkey, subvalue)
                else:
                    setattr(self, key, value)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert context to dictionary for storage"""
        return self.model_dump()
    
    def get_next_state(self) -> ConversationState:
        """Determine the next state based on current state and collected information"""
        current = self.state
        
        # Define the conversation flow
        if current == ConversationState.GREETING:
            return ConversationState.AGE_VERIFICATION
            
        elif current == ConversationState.AGE_VERIFICATION:
            if self.skin_profile.age_verified:
                return ConversationState.SKIN_TYPE
            return current  # Stay in this state until age is verified
            
        elif current == ConversationState.SKIN_TYPE:
            if self.skin_profile.skin_type:
                return ConversationState.SKIN_CONCERNS
            return current
            
        elif current == ConversationState.SKIN_CONCERNS:
            if len(self.skin_profile.concerns) > 0:
                return ConversationState.HEALTH_CHECK
            return current
            
        elif current == ConversationState.HEALTH_CHECK:
            # Check if we have the minimum health info
            if (self.health_info.is_pregnant is not None or 
                self.health_info.is_nursing is not None or
                self.health_info.planning_pregnancy is not None):
                return ConversationState.SUN_EXPOSURE
            return current
            
        elif current == ConversationState.SUN_EXPOSURE:
            if self.skin_profile.sun_exposure:
                return ConversationState.CURRENT_ROUTINE
            return current
            
        elif current == ConversationState.CURRENT_ROUTINE:
            # Check if we have at least one routine item
            if (self.routine.morning_cleanser or self.routine.morning_sunscreen or
                self.routine.evening_cleanser or self.routine.evening_moisturizer):
                return ConversationState.PRODUCT_PREFERENCES
            return current
            
        elif current == ConversationState.PRODUCT_PREFERENCES:
            if self.preferences.budget_range or len(self.preferences.requirements) > 0:
                return ConversationState.SUMMARY
            return current
            
        elif current == ConversationState.SUMMARY:
            return ConversationState.COMPLETE
            
        return current  # Default to staying in the same state

class EnhancedAgent:
    def __init__(self, settings):
        self.client = anthropic.Anthropic(api_key=settings.claude_api_key)
        self.history = []
        self.model = "claude-3-5-sonnet-20241022"
        self.context = ConversationContext()
        
        # Base system prompt with state awareness
        self.system_base = """
You are Glowbot, a smart skincare assistant who helps users create personalized skincare routines. 
Interview users naturally, asking one question at a time while maintaining a friendly, conversational tone.
Focus on gathering essential information: age, skin type, current concerns, health considerations (pregnancy, medications, allergies), and current skincare routine.

Be concise in your responses, remembering this is a chat conversation. 
Progress through information gathering logically based on the current conversation state.

Guidelines:
- Always respond in the same language as the user.
- Position Glowbot as female.
- Use follow-up questions when the user mentions something that can provide valuable information.
"""

    def _get_state_specific_prompt(self) -> str:
        """Get additional prompt instructions based on the current state"""
        
        state_prompts = {
            ConversationState.GREETING: """
You're starting a new conversation. Greet the user warmly, introduce yourself as GlowBot,
and explain that you're here to help create a personalized skincare routine.
Ask only ONE question at a time. First, confirm if they're 18 or older (for safety reasons).
""",
            ConversationState.AGE_VERIFICATION: """
You need to verify the user's age. Confirm if they are 18 or older.
If they confirm, update the collected_info to include {"skin_profile": {"age_verified": true}}
""",
            ConversationState.SKIN_TYPE: """
You need to determine the user's skin type. Ask how they would describe their skin:
- Dry (tight, flaky)
- Oily (shiny, prone to breakouts)
- Combination (oily T-zone, dry cheeks)
- Normal
- Not sure

Update collected_info with {"skin_profile": {"skin_type": "their skin type"}}
""",
            ConversationState.SKIN_CONCERNS: """
Find out the user's primary skin concerns. They can select multiple:
- Hyperpigmentation (dark spots, melasma)
- Signs of aging (fine lines, wrinkles)
- Dryness/Dehydration
- Acne
- Uneven texture
- Milia
- Rosacea
- Enlarged pores
- Other (let them describe)

Update collected_info with {"skin_profile": {"concerns": ["concern1", "concern2"]}}
""",
            ConversationState.HEALTH_CHECK: """
Find out if the user has any important health considerations:
- Are they pregnant, breastfeeding, or planning pregnancy?
- Do they have any skin sensitivities or allergies?
- Are they using prescription medications that might affect skin (retinoids, antibiotics, Accutane)?

Update health_info in collected_info accordingly.
""",
            ConversationState.SUN_EXPOSURE: """
Determine the user's sun exposure level:
- Minimal (mostly indoors)
- Moderate (1-3 hours outside)
- High (3+ hours outside)
Also ask if they spend time near windows during the day.

Update collected_info with {"skin_profile": {"sun_exposure": "their exposure level"}}
""",
            ConversationState.CURRENT_ROUTINE: """
Ask about their current skincare routine:
1. Morning Routine:
   - Cleanser?
   - Treatments/Serums?
   - Moisturizer?
   - Sunscreen?
2. Evening Routine:
   - Makeup removal method?
   - Cleanser?
   - Treatments/Serums?
   - Moisturizer?

Update routine in collected_info with what they share.
""",
            ConversationState.PRODUCT_PREFERENCES: """
Find out their product preferences:
- Budget range (budget-friendly/mid-range/high-end)
- Vegan
- Cruelty-free
- Clean/natural ingredients
- Fragrance-free
- Any other specific requirements

Update preferences in collected_info with their answers.
""",
            ConversationState.SUMMARY: """
You've collected all necessary information. Provide a summary of what you've learned about their skin:
- Skin Profile (type, concerns, sun exposure)
- Health Considerations 
- Current Routine
- Product Preferences

Ask if there's anything they'd like to add or modify.
Set action to "done" to complete the consultation.
""",
            ConversationState.COMPLETE: """
The consultation is complete. Thank the user for providing their information and let them know
you'll create a personalized skincare routine based on what they've shared.
Always set action to "done".
"""
        }
        
        return state_prompts.get(self.context.state, "")
    
    def _update_context_from_response(self, response: InterviewMessage) -> None:
        """Update the conversation context based on the collected information"""
        if response.collected_info:
            self.context.update(response.collected_info)
        
        # Progress the state based on collected information
        next_state = self.context.get_next_state()
        if next_state != self.context.state:
            self.context.state = next_state
    
    def get_claude_response(self, message: str) -> InterviewMessage:
        """Get a response from Claude with proper context management"""
        self.history.append({
            "role": "user",
            "content": message
        })
        
        # Combine base system prompt with state-specific instructions
        full_system_prompt = self.system_base + self._get_state_specific_prompt()
        
        # Add context information to the prompt
        context_section = f"""
CURRENT CONVERSATION STATE: {self.context.state}

INFORMATION COLLECTED SO FAR:
- Skin Profile: {self.context.skin_profile.model_dump_json()}
- Health Info: {self.context.health_info.model_dump_json()}
- Current Routine: {self.context.routine.model_dump_json()}
- Preferences: {self.context.preferences.model_dump_json()}

Based on the current state and information collected, ask the appropriate question to gather missing information.
In your response, include ALL new information you collect in the collected_info field.
"""
        
        full_system_prompt += context_section
        
        response = self.client.messages.create(
            model=self.model,
            max_tokens=1024,
            messages=self.history,
            tools=[{"name": "output", "input_schema": InterviewMessage.model_json_schema()}],
            tool_choice={"name": "output", "type": "tool"},
            system=full_system_prompt
        )

        assert response.content[0].type == "tool_use"

        assistant_message = response.content[0].input
        resp = InterviewMessage.model_validate(assistant_message)
        
        # Update the conversation context with newly collected information
        self._update_context_from_response(resp)
        
        # Add the assistant's response to the history
        self.history.append({
            "role": "assistant", 
            "content": resp.message
        })
        
        return resp

def run_conversation(settings):
    """Run the conversation loop with the enhanced agent"""
    agent = EnhancedAgent(settings)
    
    # Start with a greeting
    print("> Glowbot: Hello! I'm GlowBot, your personal skincare assistant. I'm here to help create a personalized skincare routine for you. First, I need to ask: are you 18 or older?")
    
    while True:
        user_input = input("You: ")
        if not user_input or user_input.lower() in ["/exit", "/quit"]:
            print("$ Exiting the conversation.")
            break
            
        print(f"> User: {user_input}")
        response = agent.get_claude_response(user_input)
        
        # Debug info
        print(f"DEBUG || State: {agent.context.state} | Action: {response.action}")
        print(f"DEBUG || Collected Info: {response.collected_info}")
        
        if response.action == InterviewAction.DONE:
            print(f"!!DONE!! Glowbot: {response.message}")
            break
            
        print(f"> Glowbot: {response.message}")
        
        # Debug current context
        print(f"DEBUG || Current Context: {agent.context.to_dict()}")